In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from datetime import date

import pandas as pd
import requests
from workalendar.europe import Netherlands

import helpers_gvb_reworked_v2 as h

%matplotlib inline
from time import time

import warnings

In [3]:
# stations to create predictions for
stations = ['Centraal Station', 'Station Zuid', 'Station Bijlmer ArenA']

#change every week
week_no = 48

In [4]:
today = pd.to_datetime("today")
today_str = str(today.year) + "-" + str(today.month) + "-" + str(today.day)
covid_url = 'https://covidtrackerapi.bsg.ox.ac.uk/api/v2/stringency/date-range/2020-09-01/' + today_str

# Main

## 1. Get data

In [5]:
print('Start loading raw data')

Start loading raw data


In [6]:
t1 = time()

herkomst_2020 = h.get_gvb_data ('Datalab_Reis_Herkomst_Uur_2020')
bestemming_2020 = h.get_gvb_data ('Datalab_Reis_Bestemming_Uur_2020')

t2 = time()
print('Completed in %s sec.' % (str(t2 - t1)))

Completed in 84.11515855789185 sec.


In [7]:
t1 = time()

herkomst_2021 = h.get_gvb_data ('Datalab_Reis_Herkomst_Uur_2021')
bestemming_2021 = h.get_gvb_data ('Datalab_Reis_Bestemming_Uur_2021')

t2 = time()
print('Completed in %s sec.' % (str(t2 - t1)))

Completed in 1.5486764907836914 sec.


In [8]:
t1 = time()

bestemming_2021 = h.get_gvb_data_json(bestemming_2021)

t2 = time()
print('Completed in %s sec.' % (str(t2 - t1)))

Completed in 41.26319241523743 sec.


In [9]:
t1 = time()

knmi_obs = h.get_knmi_data('knmi/knmi-observations/2021/**/**/*')

t2 = time()
print('Completed in %s sec.' % (str(t2 - t1)))

Completed in 1.4354498386383057 sec.


In [10]:
t1 = time()

knmi_preds = h.get_knmi_data('knmi/knmi/**/**/**/*.json.gz')

t2 = time()
print('Completed in %s sec.' % (str(t2 - t1)))

Completed in 45.58239412307739 sec.


In [11]:
covid_measures = h.get_covid_measures()

C:\Users\Antonis\Documents\UvA\DSP\DSP-project\model\helpers_gvb_reworked_v2.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  measures_rawNL["date"] = measures_rawNL.apply(


In [12]:
df_covid_filtered  = h.get_df_covid_filtered()

C:\Users\Antonis\Documents\UvA\DSP\DSP-project\model\helpers_gvb_reworked_v2.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_covid_nl['datetime'] = pd.to_datetime(df_covid_nl['dateRep']).dt.strftime('%Y-%m-%d')


In [13]:
covid_df_raw = pd.DataFrame(requests.get(url = covid_url).json()['data'])

In [14]:
holidays_data_raw = Netherlands().holidays(2019) + Netherlands().holidays(2020) + Netherlands().holidays(2021)

In [15]:
vacations_df = h.get_vacations()

In [16]:
events = h.get_events()

### 2. Prepare data

#### Pre-process data sources

In [17]:
herkomst = pd.concat([herkomst_2020, herkomst_2021])
bestemming = pd.concat([bestemming_2020, bestemming_2021])

In [18]:
# Cast 'AantalReizen' to int to sum up
bestemming['AantalReizen'] = bestemming['AantalReizen'].astype(int)
herkomst['AantalReizen'] = herkomst['AantalReizen'].astype(int)

# Remove all duplicates
bestemming.drop_duplicates(inplace=True)
herkomst.drop_duplicates(inplace=True)

# Group by station name because we are analysing per station
bestemming_grouped = bestemming.groupby(['Datum', 'UurgroepOmschrijving (van aankomst)', 'AankomstHalteNaam'], as_index=False)['AantalReizen'].sum()
herkomst_grouped = herkomst.groupby(['Datum', 'UurgroepOmschrijving (van vertrek)', 'VertrekHalteNaam'], as_index=False)['AantalReizen'].sum()

In [19]:
bestemming_herkomst = h.merge_bestemming_herkomst(bestemming_grouped, herkomst_grouped)

In [20]:
gvb_dfs = []

for station in stations:
    gvb_dfs.append(h.preprocess_gvb_data_for_modelling(bestemming_herkomst, station))

In [21]:
knmi_historical = h.preprocess_knmi_data_hour(knmi_obs)

In [22]:
knmi_forecast = h.preprocess_metpre_data(knmi_preds)

In [23]:
covid_df = h.preprocess_covid_data(covid_df_raw)

In [24]:
holiday_df = h.preprocess_holiday_data(holidays_data_raw)

#### Merge datasources

In [25]:
gvb_dfs_merged = []

for df in gvb_dfs:
    gvb_dfs_merged.append(h.merge_gvb_with_datasources(df, knmi_historical, covid_df, covid_measures, holiday_df, vacations_df, events, df_covid_filtered))

### 3. Clean data

#### Interpolate missing data

In [26]:
gvb_dfs_interpolated = []

for df in gvb_dfs_merged:
    gvb_dfs_interpolated.append(h.interpolate_missing_values(df))

C:\Users\Antonis\Documents\UvA\DSP\DSP-project\model\helpers_gvb_reworked_v2.py:465: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  checkins_interpolator.fit(X_train, y_train)
C:\Users\Antonis\Documents\UvA\DSP\DSP-project\model\helpers_gvb_reworked_v2.py:477: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  checkouts_interpolator.fit(X_train, y_train)
C:\Users\Antonis\Documents\UvA\DSP\DSP-project\model\helpers_gvb_reworked_v2.py:465: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  checkins_interpolator.fit(X_train, y_train)
C:\Users\Antonis\Documents\UvA\DSP\DSP-project\model\helpers_gvb_reworked_v2.py:477: DataConversionWarning: A column-vector y was passed when

In [27]:
gvb_dfs_final = []

for df in gvb_dfs_interpolated:

    df['check-ins'] = df['check-ins'].astype(int)
    df['check-outs'] = df['check-outs'].astype(int)
    df[['check-ins_week_ago', 'check-outs_week_ago']] = df.apply(lambda x: h.get_crowd_last_week(df, x), axis=1, result_type="expand")

    gvb_dfs_final.append(df)

#### 4. Create model dataframes

In [28]:
# Define features and targets. This is the same for all stations at the moment.
features = ['year', 'month', 'weekday', 'hour', 'holiday', 'vacation', 'planned_event', 'temperature', 'wind_speed', 'precipitation_h','global_radiation', 'cases', 'deaths'] + list(covid_measures.columns.values)
# features = ['year', 'month', 'weekday', 'hour', 'holiday', 'vacation', 'planned_event', 'temperature', 'wind_speed', 'precipitation_h','global_radiation', 'stringency', 'cases', 'deaths']

targets = ['check-ins', 'check-outs']

In [29]:
data_splits = []

for df in gvb_dfs_final:
    df = df[['datetime'] + features + targets]

    train, validation, test = h.get_train_val_test_split(df.dropna())
    data_splits.append([train, validation, test])

In [30]:
X_train_splits = []
y_train_splits = []

X_validation_splits = []
y_validation_splits = []

X_test_splits = []
y_test_splits = []

for split in data_splits:

    X_train_splits.append(split[0][features])
    y_train_splits.append(split[0][targets])

    X_validation_splits.append(split[1][features])
    y_validation_splits.append(split[1][targets])

    X_test_splits.append(split[2][features])
    y_test_splits.append(split[2][targets])

In [31]:
# Dataframes to predict cdfheck-ins and check-outs of next week
X_predict_dfs = []

for df in gvb_dfs_final:
    X_predict_dfs.append(h.get_future_df(features, df, covid_df.tail(1)['stringency'][0], covid_measures, df_covid_filtered, holiday_df, vacations_df, knmi_forecast, events))

C:\Users\Antonis\Documents\UvA\DSP\DSP-project\model\helpers_gvb_reworked_v2.py:610: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['peak_period'][df.hour.isin([7, 8, 17, 18])] = 1
C:\Users\Antonis\Documents\UvA\DSP\DSP-project\model\helpers_gvb_reworked_v2.py:610: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['peak_period'][df.hour.isin([7, 8, 17, 18])] = 1
C:\Users\Antonis\Documents\UvA\DSP\DSP-project\model\helpers_gvb_reworked_v2.py:610: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

In [45]:
X_predict_dfs[0].dropna()

,year,month,weekday,hour,holiday,vacation,planned_event,temperature,wind_speed,precipitation_h,...,RestaurantsCafesPartial,StayHomeGen,StayHomeGenPartial,StayHomeOrderPartial,StayHomeRiskG,Teleworking,TeleworkingPartial,WorkplaceClosures,WorkplaceClosuresPartial,datetime
0,2022,1,0,0,0,0,0.0,7.0,2.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2022-01-24
1,2022,1,0,1,0,0,0.0,7.0,3.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2022-01-24
2,2022,1,0,2,0,0,0.0,7.0,3.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2022-01-24
3,2022,1,0,3,0,0,0.0,7.0,3.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2022-01-24
4,2022,1,0,4,0,0,0.0,7.0,3.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2022-01-24
5,2022,1,0,5,0,0,0.0,7.0,3.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2022-01-24
6,2022,1,0,6,0,0,0.0,7.0,3.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2022-01-24
7,2022,1,0,7,0,0,0.0,7.0,2.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2022-01-24
8,2022,1,0,8,0,0,0.0,7.0,2.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2022-01-24
9,2022,1,0,9,0,0,0.0,7.0,2.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2022-01-24


### 5. Create model

In [33]:
# basic_models = []
#
# for x in range(0, len(data_splits)):
#     model_basic, r_squared_basic, mae_basic, rmse_basic = h.train_random_forest_regressor(X_train_splits[x], y_train_splits[x],
#                                                                                           X_validation_splits[x], y_validation_splits[x],
#                                                                                           None)
#     basic_models.append([model_basic, r_squared_basic, mae_basic, rmse_basic])

In [34]:
#### Tune (hyper-)parameters (not done because models currently do not improve with hyperparameter tuning)

In [35]:
# Specify hyperparameters, these could be station-specific. For now, default hyperparameter settings are being used.
centraal_station_hyperparameters = None
station_zuid_hyperparameters = None
station_bijlmer_arena_hyperparameters = None

hyperparameters = [centraal_station_hyperparameters,
                   station_zuid_hyperparameters,
                   station_bijlmer_arena_hyperparameters
                   ]

In [36]:
#tuned_models = []

#for x in range(0, len(data_splits)):
#    model_tuned, r_squared_tuned, mae_tuned, rmse_tuned = h.train_random_forest_regressor(X_train_splits[x], y_train_splits[x], 
#                                                                                          X_validation_splits[x], y_validation_splits[x], 
#                                                                                          hyperparameters[x])
#    tuned_models.append([model_tuned, r_squared_tuned, mae_tuned, rmse_tuned])

##### Improvements compared to basic model (negative is worse performance)

In [37]:
#for x in range(0, len(basic_models)):
#    print("R-squared difference", tuned_models[x][1]-basic_models[x][1])
#    print("MAE difference", tuned_models[x][2]-basic_models[x][2])
#    print("RMSE difference", tuned_models[x][3]-basic_models[x][3])

#### Train test model (including validation data)

In [38]:
test_models = []

for x in range(0, len(data_splits)):
    X_train_with_val = pd.concat([X_train_splits[x], X_validation_splits[x]])
    y_train_with_val = pd.concat([y_train_splits[x], y_validation_splits[x]])

    model_test, r_squared_test, mae_test, rmse_test = h.train_random_forest_regressor(X_train_with_val, y_train_with_val,
                                                                                          X_test_splits[x], y_test_splits[x],
                                                                                          hyperparameters[x])
    test_models.append([model_test, r_squared_test, mae_test, rmse_test])

In [39]:
test_models

[[RandomForestRegressor(random_state=1),
  0.9235309694767143,
  158.02463709677423,
  291.71866798683993],
 [RandomForestRegressor(random_state=1),
  0.8777953099640305,
  83.09130252100837,
  168.7934187018523],
 [RandomForestRegressor(random_state=1),
  0.5948012148899282,
  45.516129032258064,
  78.47413291200053]]

In [40]:
h.log_models(test_models, stations)

#### Check models on R-squared score

In [41]:
for x in range(0, len(test_models)):
    station_name = stations[x]
    r_squared = test_models[x][1]
    if r_squared < 0.7:
        warnings.warn("Model for " + station_name + " shows unexpected performance!")

<ipython-input-41-0cf3af343546>:5: UserWarning: Model for Station Bijlmer ArenA shows unexpected performance!
  warnings.warn("Model for " + station_name + " shows unexpected performance!")


#### Train final models (to make predictions)

In [42]:
final_models = []

for x in range(0, len(data_splits)):
    X_train_with_val = pd.concat([X_train_splits[x], X_validation_splits[x], X_test_splits[x]])
    y_train_with_val = pd.concat([y_train_splits[x], y_validation_splits[x], y_test_splits[x]])

    model_final = h.train_random_forest_regressor(X_train_with_val, y_train_with_val, X_test_splits[x], y_test_splits[x],
                                                  hyperparameters[x])[0]
    final_models.append(model_final)

### 6. Predict and Save

In [43]:
predictions = []
for i, model in enumerate(final_models):
    prediction = h.predict(model, X_predict_dfs[i].dropna())
    predictions.append(prediction)

for i, prediction in enumerate(predictions):
    if not os.path.exists('output/' + stations[i]):
        os.mkdir('output/' + stations[i])

    prediction.to_csv(
        ('output/{}/prediction_next_week_{}.csv'.format(stations[i], date.today().strftime("%b-%d-%Y"))))